<a href="https://colab.research.google.com/github/Chamara-g/pre-datastorm-v1-0/blob/master/pre_data_storm_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Day3 | Feature reduction**

Drive Mount

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

Train data set save to **df_train** dataframe

In [40]:
#read train data set
df_train = pd.read_csv("drive/My Drive/FYP/dataset/credit_card_default_train.csv", index_col=0)
df_train.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,
A2,1M,M,Graduate,Other,31-45,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
A3,1M,F,High School,Other,Less than 30,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,0
A4,100K,F,High School,Single,31-45,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,1
A5,200K,F,Graduate,Single,31-45,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
A6,1M,F,Graduate,Other,31-45,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,0


Train data set save to **df_test** dataframe

In [41]:
#read test data set
df_test = pd.read_csv("drive/My Drive/FYP/dataset/credit_card_default_test.csv", index_col=0)
df_test.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,,,,,,,,,,,,,,,,,,
A20170,500K,M,Other,Single,31-45,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991
A16887,200K,F,Other,Single,31-45,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620
A3430,400K,M,Other,Other,Less than 30,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060
A3696,1.5M,M,Graduate,Single,31-45,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0
A20474,100K,F,High School,Other,Less than 30,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0


String to numarical covert

In [0]:
def StringToNumData(dataFrame):
  #data set pre processing

  #convet to string balance limit to float
  dataFrame.Balance_Limit_V1 = (dataFrame.Balance_Limit_V1.replace(r'[KMB]+$', '', regex=True).astype(float) * dataFrame.Balance_Limit_V1.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M', 'B'], [10**3, 10**6, 10**9]).astype(int))
  dataFrame.head()

  #gender education marital age stringto int convert
  dataFrame.Gender.replace(['M', 'F'], [1, 0], inplace=True)
  dataFrame.EDUCATION_STATUS.replace(['Graduate', 'High School', 'Other'], [2, 1, 0], inplace=True)
  dataFrame.MARITAL_STATUS.replace(['Other', 'Single'], [1, 0], inplace=True)
  dataFrame.AGE.replace(['Less than 30', '31-45', '46-65','More than 65'], [3, 2, 1, 0], inplace=True)

  return dataFrame

In [43]:
df_train = StringToNumData(df_train) #df data convert
df_train.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,0
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,1
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [44]:
df_test = StringToNumData(df_test) #df_test data convert
df_test.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,,,,,,,,,,,,,,,,,,
A20170,500000.0,1,0,0,2,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991
A16887,200000.0,0,0,0,2,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620
A3430,400000.0,1,0,1,3,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060
A3696,1500000.0,1,2,0,2,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0
A20474,100000.0,0,1,1,3,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0


Features and labels save to X and y

In [0]:
X = df_train.drop("NEXT_MONTH_DEFAULT",1)   #Feature Matrix
y = df_train["NEXT_MONTH_DEFAULT"]          #Target Variable

In [0]:
X_test = df_test

**Find Gradient of DUE_AMT**

take y axis as [1,2,3,4,5,6] and take gradient of due_amt features into on feature name **DUE_AMT_GRD**

In [46]:
DUE_X = X[["DUE_AMT_JULY", "DUE_AMT_AUG", "DUE_AMT_SEP", "DUE_AMT_OCT", "DUE_AMT_NOV", "DUE_AMT_DEC"]]
DUE_X.head()

,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC
Client_ID,,,,,,
A2,3248,3389,6004,39418,162772,-13982
A3,353351,151818,26948,43530,80811,124590
A4,16681,16082,15477,0,0,0
A5,90457,92848,95193,97309,100353,102740
A6,429556,419466,429785,435354,445271,453899


In [0]:
from statistics import mean

def best_fit_slope(xs,ys):
    m = (((mean(xs)*mean(ys)) - mean(xs*ys)) /
         ((mean(xs)**2) - mean(xs**2)))
    return m

DUE_AMT_GRD feature added to Feature List

In [0]:
def findGradientDf(dataframeX, choice):
  # Create an empty list 
  Row_list =[] 
    
  # Iterate over each row 
  for index, rows in dataframeX.iterrows(): 
      # Create list for the current row 
      if(choice == 1): #for due amount
        my_list =[rows.DUE_AMT_JULY, rows.DUE_AMT_AUG, rows.DUE_AMT_SEP, rows.DUE_AMT_OCT, rows.DUE_AMT_NOV, rows.DUE_AMT_DEC]
      elif(choice == 2): #for paid amount
        my_list =[rows.PAID_AMT_JULY, rows.PAID_AMT_AUG, rows.PAID_AMT_SEP, rows.PAID_AMT_OCT, rows.PAID_AMT_NOV, rows.PAID_AMT_DEC]
      xs = np.array([1,2,3,4,5,6], dtype=np.float64)
      ys = np.array(my_list, dtype=np.float64)

      # append the list to the final list 
      Row_list.append(best_fit_slope(xs,ys)) 
    
  return Row_list

In [54]:
X["DUE_AMT_GRD"] = findGradientDf(DUE_X, 1)
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,12154.657143
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,-38292.685714
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,-4203.657143
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,2458.457143
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,5848.542857


**Find Gradient of PAID_AMT**

take y axis as [1,2,3,4,5,6] and take gradient of paid_amt features into on feature name **PAID_AMT_GRD**

In [55]:
PAID_X = X[["PAID_AMT_JULY", "PAID_AMT_AUG", "PAID_AMT_SEP", "PAID_AMT_OCT", "PAID_AMT_NOV", "PAID_AMT_DEC"]]
PAID_X.head()

,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,
A2,3437,6004,39418,162772,0,538165
A3,151818,46200,43530,80811,942,33666
A4,0,0,0,0,0,0
A5,3855,3890,3696,4620,4049,3918
A6,0,20790,16170,17325,16401,17325


In [56]:
X["PAID_AMT_GRD"] = findGradientDf(PAID_X, 2)
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,12154.657143,79399.485714
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,-38292.685714,-19692.942857
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,-4203.657143,-0.000000
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,2458.457143,49.028571
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,5848.542857,2131.800000


**Find sum of PAY**

Take sum of pay col and make one feature call **PAY_GRD**

In [57]:
PAY_X = X[["PAY_JULY", "PAY_AUG", "PAY_SEP", "PAY_OCT", "PAY_NOV", "PAY_DEC"]]
PAY_X.head()

,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC
Client_ID,,,,,,
A2,-1,-1,-1,-1,-1,-1
A3,0,-1,-1,-1,-1,0
A4,4,3,2,2,-2,-2
A5,2,0,0,0,0,0
A6,2,2,0,0,0,0


In [58]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in PAY_X.iterrows(): 
    # Create list for the current row 
    my_list =[rows.PAY_JULY, rows.PAY_AUG, rows.PAY_SEP, rows.PAY_OCT, rows.PAY_NOV, rows.PAY_DEC] 
    Row_list.append(sum(my_list))

X["PAY_GRD"] = Row_list
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD,PAY_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,12154.657143,79399.485714,-6
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,-38292.685714,-19692.942857,-4
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,-4203.657143,-0.000000,7
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,2458.457143,49.028571,2
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,5848.542857,2131.800000,4


**Find Gradient of PAID AND DUE**

Take mean of PAID_AMT_GRD and DUE_AMT_GRD to **DUE_PAID_GRD**

In [59]:
PAY_DUE_X = X[["DUE_AMT_GRD", "PAID_AMT_GRD"]]
PAY_DUE_X.head()

,DUE_AMT_GRD,PAID_AMT_GRD
Client_ID,,
A2,12154.657143,79399.485714
A3,-38292.685714,-19692.942857
A4,-4203.657143,-0.000000
A5,2458.457143,49.028571
A6,5848.542857,2131.800000


In [61]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in PAY_DUE_X.iterrows(): 
    # Create list for the current row 
    my_list =[rows.DUE_AMT_GRD, rows.PAID_AMT_GRD] 
    Row_list.append(sum(my_list)/2)

X["DUE_PAID_GRD"] = Row_list
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD,PAY_GRD,PAY_DUE_GRD,DUE_PAID_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,12154.657143,79399.485714,-6,45777.071429,45777.071429
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,-38292.685714,-19692.942857,-4,-28992.814286,-28992.814286
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,-4203.657143,-0.000000,7,-2101.828571,-2101.828571
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,2458.457143,49.028571,2,1253.742857,1253.742857
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,5848.542857,2131.800000,4,3990.171429,3990.171429


In [0]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

In [0]:
#Testing and traing split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

Feature importance calculate using new features

In [64]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(X.columns, clf.feature_importances_):
    print(feature)

('Balance_Limit_V1', 0.031121182431696648)
('Gender', 0.011503340151323685)
('EDUCATION_STATUS', 0.01707254818243888)
('MARITAL_STATUS', 0.01075968392764675)
('AGE', 0.019078861990288853)
('PAY_JULY', 0.08187504791582319)
('PAY_AUG', 0.0320518358509797)
('PAY_SEP', 0.02094231841458811)
('PAY_OCT', 0.0170766270802282)
('PAY_NOV', 0.015598441071090999)
('PAY_DEC', 0.013562256525787695)
('DUE_AMT_JULY', 0.045792829435540375)
('DUE_AMT_AUG', 0.042565527045642704)
('DUE_AMT_SEP', 0.04161534919065069)
('DUE_AMT_OCT', 0.04087445164684069)
('DUE_AMT_NOV', 0.039980299024827866)
('DUE_AMT_DEC', 0.03905490950930638)
('PAID_AMT_JULY', 0.041640469261235526)
('PAID_AMT_AUG', 0.03925600966126395)
('PAID_AMT_SEP', 0.03920122544280287)
('PAID_AMT_OCT', 0.03630023654791795)
('PAID_AMT_NOV', 0.03533926350219735)
('PAID_AMT_DEC', 0.0358236391369192)
('DUE_AMT_GRD', 0.047775463153716495)
('PAID_AMT_GRD', 0.04813347857193593)
('PAY_GRD', 0.06252974390741745)
('PAY_DUE_GRD', 0.04678304355124631)
('DUE_PAID_G

Save new data set into csv

In [68]:
X['NEXT_MONTH_DEFAULT'] = y #add labels
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD,PAY_GRD,PAY_DUE_GRD,DUE_PAID_GRD,NEXT_MONTH_DEFAULT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,12154.657143,79399.485714,-6,45777.071429,45777.071429,0
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,-38292.685714,-19692.942857,-4,-28992.814286,-28992.814286,0
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,-4203.657143,-0.000000,7,-2101.828571,-2101.828571,1
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,2458.457143,49.028571,2,1253.742857,1253.742857,1
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,5848.542857,2131.800000,4,3990.171429,3990.171429,0


In [0]:
X.to_csv('drive/My Drive/FYP/dataset/credit_card_data_train_pre.csv', encoding='utf-8', mode='w', index=True)

New test data set create using new features that we create

In [71]:
DUE_X_test = X_test[["DUE_AMT_JULY", "DUE_AMT_AUG", "DUE_AMT_SEP", "DUE_AMT_OCT", "DUE_AMT_NOV", "DUE_AMT_DEC"]]
X_test["DUE_AMT_GRD"] = findGradientDf(DUE_X_test, 1)

PAID_X_test = X_test[["PAID_AMT_JULY", "PAID_AMT_AUG", "PAID_AMT_SEP", "PAID_AMT_OCT", "PAID_AMT_NOV", "PAID_AMT_DEC"]]
X_test["PAID_AMT_GRD"] = findGradientDf(PAID_X_test, 2)

PAY_X_test = X_test[["PAY_JULY", "PAY_AUG", "PAY_SEP", "PAY_OCT", "PAY_NOV", "PAY_DEC"]]

# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in PAY_X_test.iterrows(): 
    # Create list for the current row 
    my_list =[rows.PAY_JULY, rows.PAY_AUG, rows.PAY_SEP, rows.PAY_OCT, rows.PAY_NOV, rows.PAY_DEC] 
    Row_list.append(sum(my_list))

X_test["PAY_GRD"] = Row_list

PAY_DUE_X_test = X_test[["DUE_AMT_GRD", "PAID_AMT_GRD"]]
Row_list =[] 
  
# Iterate over each row 
for index, rows in PAY_DUE_X_test.iterrows(): 
    # Create list for the current row 
    my_list =[rows.DUE_AMT_GRD, rows.PAID_AMT_GRD] 
    Row_list.append(sum(my_list)/2)

X_test["DUE_PAID_GRD"] = Row_list
X_test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD,PAY_GRD,PAY_DUE_GRD,DUE_PAID_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A6327,200000.0,1,0,0,2,-1,-1,-1,-1,-1,-1,2303,93086,15239,1536,23370,0,93086,15239,8004,23370,0,1536,-6696.171429,-13945.742857,-6,-10320.957143,-10320.957143
A12869,200000.0,0,0,1,3,0,0,0,0,0,-1,104211,61899,44121,33726,20790,4620,3363,2772,0,0,4620,12705,-18047.914286,1492.971429,-1,-8277.471429,-8277.471429
A7703,1000000.0,0,1,1,2,-1,-1,-1,-1,-1,-1,11139,536,5209,5881,6122,14521,538,5234,5909,6152,14560,28,981.142857,733.457143,-6,857.300000,857.300000
A4150,400000.0,0,1,0,2,0,0,-2,-2,-2,-2,36318,0,0,0,0,0,0,0,0,0,0,0,-5188.285714,-0.000000,-8,-2594.142857,-2594.142857
A8788,1000000.0,1,0,0,2,0,0,0,0,0,0,88547,146445,154472,160173,72823,71289,69300,10857,6930,11550,11550,16170,-8613.000000,-7398.600000,0,-8005.800000,-8005.800000


Save csv file

In [0]:
X_test.to_csv('drive/My Drive/FYP/dataset/credit_card_data_test_pre.csv', encoding='utf-8', mode='w', index=True)